In [ ]:
import pandas as pd 
import numpy as np

## نورمال سازی دیتا ها مناسب شدن برای تحلیل احساسات

## دیتای توویتر

In [ ]:

df = pd.read_csv("D:\code\projects\EmoLex(امولکس)\Data\MainData\Data.csv")


unique_classes = df['label'].unique()
class_count = df['label'].nunique()

print("کلاس‌های یکتا:", unique_classes)
print("تعداد کلاس‌ها:", class_count)

# ذخیره نسخه تمیزشده
df.to_csv("Data_clean.csv", index=False)


## یکه کردن کلاس لیبل 

In [ ]:
import pandas as pd


df = pd.read_csv("D:\code\projects\EmoLex(امولکس)\Data\MainData\Data_clean.csv", sep=None, engine="python", encoding="utf-8-sig")


label_col = df.columns[-1]


df[label_col] = (
    df[label_col]
    .astype(str).str.strip().str.lower()
    .map(mapping)
)


unmapped = df[label_col].isna()
if unmapped.any():
    print("Warning: Unknown labels found at rows:", df[unmapped].index.tolist())


df.to_csv("Data.csv", index=False, encoding="utf-8-sig")
print("Labels converted to: happy, sad, neutral (saved in Data.csv).")


## اپر کیس کردن کلاس لیبل

In [ ]:
import pandas as pd

file_path = r"D:\code\projects\EmoLex(امولکس)\Data\MainData\Data.csv"

df = pd.read_csv(file_path)

df["label"] = df["label"].str.upper()

df.to_csv(file_path, index=False, encoding='utf-8-sig')



## اسنپ فود

## بک اپ گیری از دیتا در پوشه دیتا های اصلی و شمارش کلاس ها 

In [ ]:
import pandas as pd
import csv
import shutil
from pathlib import Path
import os

file_path = Path("D:\code\projects\EmoLex(امولکس)\Data\اسنپ فود\Snappfood - Sentiment AnalysisُSnappfoodData.csv")
backup_path = Path(os.getcwd()) / "snappfood.csv"
shutil.copy(file_path, backup_path)

detected_sep = None
with open(file_path, 'r', encoding='utf-8-sig', errors='replace') as f:
    sample = ''.join([next(f) for _ in range(200)])
    try:
        dialect = csv.Sniffer().sniff(sample, delimiters=[',',';','\t','|'])
        detected_sep = dialect.delimiter
    except Exception:
        pass

candidates = [s for s in [detected_sep, ',', ';', '\t', '|'] if s]
df = None
for sep in candidates:
    try:
        df = pd.read_csv(file_path, sep=sep, engine='python', encoding='utf-8-sig', quoting=csv.QUOTE_MINIMAL)
        break
    except:
        pass

if df is not None and "label" in df.columns:
    print(df["label"].unique())
    print(df["label"].nunique())
    print(df["label"].value_counts())


## حدف کلاس ها و  ستون هایی که به درد نمیخورن 

In [ ]:
import pandas as pd

file_path = r"D:\code\projects\EmoLex(امولکس)\Data\MainData\snappfood.csv"

# Read as tab-separated
df = pd.read_csv(file_path, sep="\t", encoding="utf-8-sig")

# Drop label_id if it exists
if "label_id" in df.columns:
    df.drop(columns=["label_id"], inplace=True)

# Save back to same file
df.to_csv(file_path, sep="\t", index=False, encoding="utf-8-sig")


## شمارش کلاس

In [ ]:
import pandas as pd
import csv

file_path = r"D:\code\projects\EmoLex(امولکس)\Data\MainData\snappfood.csv"

def load_tsv_robust(path: str) -> pd.DataFrame:
    try:
        return pd.read_csv(path, sep="\t", engine="python", encoding="utf-8-sig",
                           quoting=csv.QUOTE_MINIMAL)
    except Exception:
        try:
            return pd.read_csv(path, sep="\t", engine="python", encoding="utf-8-sig",
                               quoting=csv.QUOTE_NONE, escapechar="\\")
        except Exception:
            with open(path, "r", encoding="utf-8-sig", errors="replace") as f:
                header = f.readline().rstrip("\r\n")
                cols = header.split("\t")
                exp = len(cols)
                rows = []
                for line in f:
                    parts = line.rstrip("\r\n").split("\t", maxsplit=exp-1)
                    if len(parts) < exp:
                        parts += [""] * (exp - len(parts))
                    rows.append(parts)
            df = pd.DataFrame(rows, columns=[c.replace("\ufeff", "").strip() for c in cols])
            return df

df = load_tsv_robust(file_path)

# normalize column names just in case
df.columns = [c.strip() for c in df.columns]

if "label" in df.columns:
    print("Unique classes:", df["label"].unique())
    print("Number of classes:", df["label"].nunique())
    print("\nCount per class:")
    print(df["label"].value_counts())
else:
    print("No 'label' column found. Columns are:", df.columns.tolist())


## تیدبل به کلاس یکه

In [ ]:
import pandas as pd
import re

file_path = r"D:\code\projects\EmoLex(امولکس)\Data\MainData\snappfood.csv"

# Load as tab-separated
df = pd.read_csv(file_path, sep="\t", engine="python", encoding="utf-8-sig", on_bad_lines="skip")

# Normalize column names
df.columns = [c.strip() for c in df.columns]

# Remove rows where label is exactly "0" or "1"
df = df[~df["label"].isin(["0", "1"])]

# Clean the label column
df["label"] = df["label"].apply(lambda x: re.sub(r'\t.*', '', str(x)).strip())

# Save back to same file
df.to_csv(file_path, sep="\t", index=False, encoding="utf-8-sig")

# Check the unique classes after cleaning
print(df["label"].unique())


## حذف کردن رکورد هایی که nan هستند

In [ ]:
import pandas as pd
import re
import numpy as np

file_path = r"D:\code\projects\EmoLex(امولکس)\Data\MainData\snappfood.csv"

df = pd.read_csv(file_path, sep="\t", engine="python", encoding="utf-8-sig", on_bad_lines="skip")
df.columns = [c.strip() for c in df.columns]

# Remove rows where label is exactly "0" or "1"
df = df[~df["label"].isin(["0", "1"])]

# Clean up tabs/quotes
df["label"] = df["label"].apply(lambda x: re.sub(r'\t.*', '', str(x)).strip())

# Convert "nan" string to real NaN
df["label"] = df["label"].replace("nan", np.nan)

# Drop rows where label is NaN
df = df.dropna(subset=["label"])

# Save back to same file
df.to_csv(file_path, sep="\t", index=False, encoding="utf-8-sig")

print(df["label"].unique())


## دیتا ست فارسی 

In [ ]:
import pandas as pd

# مسیر فایل CSV
file_path = r"D:\code\projects\EmoLex(امولکس)\Data\MainData\IrainianDataSet.csv"

# خواندن فایل
df = pd.read_csv(file_path)

# پیدا کردن ستون لیبل
label_col = None
for col in df.columns:
    if col.lower() in ["label", "labels", "emotion", "emotions"]:
        label_col = col
        break

if label_col:
    # شمارش تعداد هر کلاس
    class_counts = df[label_col].value_counts()
    print("تعداد هر کلاس:")
    print(class_counts)
else:
    print("ستون لیبل در این دیتاست پیدا نشد.")


## دستا ست ایرانی دوم

In [ ]:
import pandas as pd
from pathlib import Path

# --- Paths ---
src = r"D:\code\projects\EmoLex(امولکس)\Data\MainData\newdataset.csv"
out_main = r"D:\code\projects\EmoLex(امولکس)\Data\MainData\combined_fa_named.csv"
out_backup = r"D:\code\projects\EmoLex(امولکس)\Data\MainData\IranianDataSet2.csv"

# Make sure the source exists
if not Path(src).exists():
    raise FileNotFoundError(f"Source file not found: {src}")

# Read only first two columns and name them directly
df = pd.read_csv(
    src,
    header=None,          # treat file as no header row
    usecols=[0, 1],       # keep only col 0 and 1
    names=["comments", "label"],
    dtype=str,            # keep everything as string (safer for text)
    encoding="utf-8"      # adjust if your file uses another encoding
)

# Optional cleaning (safe to keep)
df["comments"] = df["comments"].astype(str).str.strip()
df["label"] = df["label"].astype(str).str.strip()

# Save outputs
df.to_csv(out_main, index=False, encoding="utf-8-sig")
df.to_csv(out_backup, index=False, encoding="utf-8-sig")

print(f"Saved: {out_main}")
print(f"Backup: {out_backup}")
print("Shape:", df.shape)
print("Head:\n", df.head())


## شمارش کلاس و حدف در صورت نیاز

In [ ]:

df = pd.read_csv("D:\code\projects\EmoLex(امولکس)\Data\MainData\IranianDataSet2.csv")
result = df["label"].value_counts()
print(result)


## حذف ستون های خالی و به درد نخور

In [ ]:
file_path = r"D:\code\projects\EmoLex(امولکس)\Data\MainData\IranianDataSet2.csv"

df = pd.read_csv(file_path)

df = df.dropna(subset=["label"])

df = df[df["label"].str.lower() != "other"]

df = df.reset_index(drop=True)

df.to_csv(file_path, index=False, encoding="utf-8-sig")

## گرفتن شمارش کلاس دوباره

In [ ]:

file_path = r"D:\code\projects\EmoLex(امولکس)\Data\MainData\IranianDataSet2.csv"

df = pd.read_csv(file_path)

class_counts = df["label"].value_counts()

num_classes = class_counts.shape[0]

print("📊 Class counts after cleaning:")
print(class_counts)
print(f"\n🔢 Number of unique classes: {num_classes}")


##  دیتا ست ایرانی سوم

In [ ]:
df = pd.read_csv("D:\code\projects\EmoLex(امولکس)\Data\MainData\iraniandataset3.csv")
result = df["label"].value_counts()
print(result)

## حذف دیتا ها OTHER

In [ ]:
file_path = r"D:\code\projects\EmoLex(امولکس)\Data\MainData\IranianDataSet3.csv"

df = pd.read_csv(file_path)

df = df.dropna(subset=["label"])

df = df[df["label"].str.lower() != "other"]

df = df.reset_index(drop=True)

df.to_csv(file_path, index=False, encoding="utf-8-sig")

## شمارش دوباره

In [ ]:
df = pd.read_csv("D:\code\projects\EmoLex(امولکس)\Data\MainData\iraniandataset3.csv")
result = df["label"].value_counts()
print(result)

## 📦  دیجی‌کالا: خوشه‌بندی داده‌ها

به دلیل **نداشتن برچسب** در داده‌ها، از روش **کلاسترینگ** برای خوشه‌بندی استفاده می‌کنیم.  
پس از انجام خوشه‌بندی، به هر خوشه یک **برچسب مناسب** اختصاص داده می‌شود.  

---

### 📌 الگوریتم مورد استفاده:
- **K-Means Clustering**

---

### 📊 حجم داده‌ها:
- **۷ میلیون رکورد**

---

### 🔄 مراحل کلی کار:
1. **پیش‌پردازش داده‌ها** (پاک‌سازی و آماده‌سازی)
2. **اجرای الگوریتم K-Means**
3. **تحلیل خوشه‌ها**
4. **برچسب‌گذاری خوشه‌ها** (Labeling)
5. **ذخیره‌سازی نتایج**

---

💡 نکته: به دلیل حجم بالای داده‌ها، استفاده از **پردازش موازی** و **بهینه‌سازی حافظه** توصیه می‌شود.


## 🔹 مرحله ۱: حذف ستون‌های اضافه

قبل از اجرای الگوریتم **K-Means**، باید فقط ستون‌های مورد نیاز برای خوشه‌بندی را نگه داریم  
و بقیه ستون‌ها (که در فرآیند خوشه‌بندی استفاده نمی‌شوند) را حذف کنیم.

In [ ]:
import pandas as pd

# مسیر فایل
file_path = r"D:\code\projects\EmoLex(امولکس)\Data\دیجی کالا\digikala-comments.csv"

# خواندن CSV
df = pd.read_csv(file_path, encoding="utf-8-sig")

# ستون‌هایی که می‌خواهیم نگه داریم
keep_cols = ["body", "title", "advantages", "disadvantages", "rate", "recommendation_status"]

# حذف ستون‌های اضافی
df = df[keep_cols]

# ذخیره روی همان فایل (بازنویسی)
df.to_csv(file_path, index=False, encoding="utf-8-sig")

print("✅ فایل تمیز شد و فقط ستون‌های اصلی باقی ماندند.")
print("ستون‌های فعلی:", df.columns.tolist())
print("تعداد ردیف‌ها:", len(df))


## 🔹 مرحله ۲: نرمال‌سازی و سبک‌سازی داده‌ها

بعد از حذف ستون‌های غیرضروری، باید داده‌ها را **نرمال‌سازی** کنیم تا همه ویژگی‌ها در یک بازه‌ی یکسان قرار بگیرند.  
همچنین برای کاهش مصرف حافظه و سرعت‌بخشیدن به اجرای الگوریتم K-Means، **نوع داده‌ها (Data Types)** را سبک‌سازی می‌کنیم.

---

In [ ]:
import pandas as pd
import re
import unicodedata
from pathlib import Path

# ---------- مسیرها ----------
SRC = r"D:\code\projects\EmoLex(امولکس)\Data\دیجی کالا\digikala-comments.csv"
DST = r"D:\code\projects\EmoLex(امولکس)\Data\دیجی کالا\digikala-comments-clean-light.csv"

# ---------- تنظیمات سبک‌سازی ----------
CHUNK = 200_000          # بسته به RAM تغییر بده
ENC   = "utf-8-sig"
KEEP_META = True         # اگر فقط text می‌خواهی، False کن

# ---------- الگوها ----------
URL_RE      = re.compile(r"https?://\S+|www\.\S+")
MENTION_RE  = re.compile(r"[@#]\w+")
EMOJI_RE    = re.compile("[" "\U0001F300-\U0001FAFF" "\U00002700-\U000027BF" "\U0001F1E6-\U0001F1FF" "]+", flags=re.UNICODE)
NON_TEXT_RE = re.compile(r"[^\w\s\u0600-\u06FF]", flags=re.UNICODE)
MULTISPACE  = re.compile(r"\s+")

def strip_diacritics(s: str) -> str:
    return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

def norm_fa(t: str) -> str:
    if not isinstance(t, str): return ""
    t = t.strip()
    if not t: return ""
    t = URL_RE.sub(" ", t)
    t = MENTION_RE.sub(" ", t)
    t = EMOJI_RE.sub(" ", t)
    t = t.replace("ي","ی").replace("ى","ی").replace("ك","ک").replace("ـ"," ")
    t = strip_diacritics(t)
    t = NON_TEXT_RE.sub(" ", t)
    t = t.lower()
    t = MULTISPACE.sub(" ", t).strip()
    return t

# ---------- پردازش چانکی + سبک‌سازی ----------
src = Path(SRC)
if not src.exists():
    raise FileNotFoundError(f"Input not found: {src}")

Path(DST).unlink(missing_ok=True)
first_write = True
total_in = total_out = 0

# برای کاهش حافظه: فقط ستون‌های موردنیاز را می‌خوانیم اگر حاضر باشند
base_cols = ["title","advantages","disadvantages","body","rate","recommendation_status"]
read_sample = pd.read_csv(SRC, nrows=5, encoding=ENC, on_bad_lines="skip")
available = [c for c in base_cols if c in read_sample.columns]
usecols = available if available else None  # اگر هدر متفاوت بود، همه را بخوان و بعد هندل کن

for chunk in pd.read_csv(SRC, chunksize=CHUNK, encoding=ENC, on_bad_lines="skip", usecols=usecols):
    total_in += len(chunk)

    # تضمین وجود ستون‌های متنی
    for col in ["title","advantages","disadvantages","body"]:
        if col not in chunk.columns:
            chunk[col] = ""
        chunk[col] = chunk[col].astype(str).map(norm_fa)

    # ساخت متن نهایی
    chunk["text"] = (chunk["title"] + " " + chunk["advantages"] + " " +
                     chunk["disadvantages"] + " " + chunk["body"]).str.strip()

    # نگه‌داشتن فقط ستون‌های لازم (سبک‌سازی)
    if KEEP_META:
        # rate را کم‌حجم کن، recommendation_status را category کن
        if "rate" in chunk.columns:
            # تلاش برای تبدیل امن به عدد کوچک
            chunk["rate"] = pd.to_numeric(chunk["rate"], errors="coerce").astype("float32")
            # می‌توانی اگر صحیح بود، به int8 هم تبدیل کنی:
            # chunk["rate"] = pd.to_numeric(chunk["rate"], errors="coerce").fillna(-1).astype("int8")
        else:
            chunk["rate"] = pd.NA

        if "recommendation_status" in chunk.columns:
            chunk["recommendation_status"] = chunk["recommendation_status"].astype("string").str.strip().str.lower().astype("category")
        else:
            chunk["recommendation_status"] = pd.Categorical([])

        out = chunk[["text","rate","recommendation_status"]].copy()
    else:
        out = chunk[["text"]].copy()

    # حذف سطرهای خالی
    out = out[out["text"] != ""]
    # حذف رکوردهای تکراری
    out = out.drop_duplicates()

    total_out += len(out)

    out.to_csv(DST, mode="a", index=False, encoding="utf-8-sig", header=first_write)
    first_write = False

print(f"✅ Done. Input rows: {total_in:,} -> Kept after cleaning: {total_out:,}")
print(f"Saved to: {DST}")


## 🔹 مرحله ۳: بردارسازی متن + کاهش‌بُعد (برای ۷ میلیون رکورد)

در این مرحله متن‌ها را به بردارهای عددی تبدیل می‌کنیم (**Text Vectorization**) و سپس برای کارایی و سرعت بیشتر، بُعد ویژگی‌ها را کاهش می‌دهیم (**Dimensionality Reduction**).

---

### 🎯 هدف
- تولید ویژگی‌های عددی از متن برای خوشه‌بندی با K-Means
- کم‌کردن ابعاد برای کاهش مصرف RAM/زمان و بهبود کیفیت خوشه‌ها

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.random_projection import SparseRandomProjection
from scipy import sparse
from pathlib import Path

FILE = r"D:\code\projects\EmoLex(امولکس)\Data\دیجی کالا\digikala-comments-clean-light.csv"
OUT_DIR = Path(r"D:\code\projects\EmoLex(امولکس)\Data\دیجی کالا\vec_out")
OUT_DIR.mkdir(parents=True, exist_ok=True)

CHUNK = 200_000
n_features = 2**20   # ~1,048,576 ویژگی؛ در صورت کمبود RAM → 2**18
proj_dim   = 256     # ابعاد نهایی بعد از کاهش بُعد

hv = HashingVectorizer(
    n_features=n_features,
    alternate_sign=False,
    norm="l2",
    lowercase=False,
)

# فیت SRP فقط یک بار روی یک/دو چانک اول
srp = SparseRandomProjection(n_components=proj_dim, random_state=42)

# مرحله 1: fit SRP روی یک چانک نمونه
sample = pd.read_csv(FILE, nrows=CHUNK, encoding="utf-8-sig")
X_sample = hv.transform(sample["text"].astype(str))
srp.fit(X_sample)
del sample, X_sample

# مرحله 2: تبدیل و ذخیره‌ی چانکی
part = 0
total = 0
for chunk in pd.read_csv(FILE, chunksize=CHUNK, encoding="utf-8-sig"):
    X_hash = hv.transform(chunk["text"].astype(str))  # sparse
    X_proj = srp.transform(X_hash)                    # sparse-ish matrix
    # ذخیره هر پارت به npz
    out_npz = OUT_DIR / f"X_proj_part{part:05d}.npz"
    sparse.save_npz(out_npz, X_proj.tocsr())
    total += len(chunk)
    part += 1
    print(f"Saved {out_npz} | rows so far: {total:,}")

print("✅ Done. Parts saved in:", OUT_DIR)
print("Each part has shape roughly: (chunk_size, proj_dim)")


Saved D:\code\projects\EmoLex(امولکس)\Data\دیجی کالا\vec_out\X_proj_part00000.npz | rows so far: 200,000
Saved D:\code\projects\EmoLex(امولکس)\Data\دیجی کالا\vec_out\X_proj_part00001.npz | rows so far: 371,746
✅ Done. Parts saved in: D:\code\projects\EmoLex(امولکس)\Data\دیجی کالا\vec_out
Each part has shape roughly: (chunk_size, proj_dim)


ادغام دو دیتای ماتریسی

In [9]:
from pathlib import Path
import glob
from scipy import sparse

VEC_DIR = Path(r"D:\code\projects\EmoLex(امولکس)\Data\دیجی کالا\vec_out")
OUT_ONE = VEC_DIR / "X_proj_all.npz"

parts = sorted(glob.glob(str(VEC_DIR / "X_proj_part*.npz")))
if not parts:
    raise FileNotFoundError("No part files found.")

# بارگذاری و ادغام (برای ابعاد ~۳۷۱k×۲۵۶ مشکلی از نظر RAM نیست)
mats = [sparse.load_npz(p) for p in parts]
X_all = sparse.vstack(mats, format="csr")

sparse.save_npz(OUT_ONE, X_all)
print("✅ Saved single file:", OUT_ONE)
print("Shape:", X_all.shape)


✅ Saved single file: D:\code\projects\EmoLex(امولکس)\Data\دیجی کالا\vec_out\X_proj_all.npz
Shape: (371746, 256)


ویژوالایز کردن اختیاری

In [7]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy import sparse

# --------- مسیرها را تنظیم کن ---------
CLEAN_CSV = r"D:\code\projects\EmoLex(امولکس)\Data\دیجی کالا\digikala-comments-clean-light.csv"  # شامل ستون text (و شاید rate/recommendation_status)
VEC_NPZ   = r"D:\code\projects\EmoLex(امولکس)\Data\دیجی کالا\vec_out\X_proj_all.npz"            # ویژگی‌های کم‌بعدی (SRP)، اگر داری
OUT_DIR   = Path(r"D:\code\projects\EmoLex(امولکس)\Data\دیجی کالا\viz")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ---------- فونت فارسی (اختیاری) ----------
# اگر با حروف فارسی مشکل داشتی، یک فونت نصب‌شده مثل Tahoma/Vazirmatn را فعال کن:
# import matplotlib
# matplotlib.rcParams['font.family'] = 'Tahoma'  # یا 'Vazirmatn'

# ---------- 1) خواندن داده‌ی متنی ----------
df = pd.read_csv(CLEAN_CSV, encoding="utf-8-sig")
assert "text" in df.columns, "ستون 'text' باید وجود داشته باشد."

# ---------- 2) طول متن‌ها ----------
df["text_len_chars"] = df["text"].astype(str).str.len()
df["text_len_tokens"] = df["text"].astype(str).str.split().apply(len)

# هیستوگرام طول کاراکتری
plt.figure(figsize=(8,5))
plt.hist(df["text_len_chars"].clip(upper=2000), bins=50)  # کلیپ برای جلوگیری از دُم خیلی بلند
plt.title("Histogram of text length (chars)")
plt.xlabel("chars")
plt.ylabel("count")
plt.tight_layout()
plt.savefig(OUT_DIR / "hist_text_len_chars.png", dpi=150)
plt.close()

# هیستوگرام طول توکنی
plt.figure(figsize=(8,5))
plt.hist(df["text_len_tokens"].clip(upper=300), bins=50)
plt.title("Histogram of text length (tokens)")
plt.xlabel("tokens")
plt.ylabel("count")
plt.tight_layout()
plt.savefig(OUT_DIR / "hist_text_len_tokens.png", dpi=150)
plt.close()

# ---------- 3) توزیع rate (اگر وجود داشته باشد) ----------
if "rate" in df.columns:
    # سعی برای تبدیل به عدد
    rate_num = pd.to_numeric(df["rate"], errors="coerce")
    rate_num = rate_num.dropna()
    if not rate_num.empty:
        plt.figure(figsize=(8,5))
        plt.hist(rate_num, bins=20)
        plt.title("Histogram of rate")
        plt.xlabel("rate")
        plt.ylabel("count")
        plt.tight_layout()
        plt.savefig(OUT_DIR / "hist_rate.png", dpi=150)
        plt.close()

# ---------- 4) شمارش recommendation_status (اگر وجود داشته باشد) ----------
if "recommendation_status" in df.columns:
    counts = df["recommendation_status"].astype(str).str.strip().value_counts()
    plt.figure(figsize=(8,5))
    counts.plot(kind="bar")
    plt.title("Counts of recommendation_status")
    plt.xlabel("status")
    plt.ylabel("count")
    plt.tight_layout()
    plt.savefig(OUT_DIR / "bar_recommendation_status.png", dpi=150)
    plt.close()

# ---------- 5) پرکاربردترین کلمات (Top-N terms) ----------
# توجه: چون متن‌ها قبلاً نرمال شده‌اند، CountVectorizer ساده کافی است.
persian_stopwords = [
    "و","به","در","از","که","این","را","با","برای","است","می","اما","یا","اگر","نه",
    "من","تو","او","ما","شما","آنها","یک","هیچ","هر","بود","شود","هم","همه"
]
vec = CountVectorizer(max_features=50000, stop_words=persian_stopwords)
# برای سرعت، نمونه‌گیری (اگر دیتاست خیلی بزرگ است):
sample_n = min(200_000, len(df))
sample_texts = df["text"].astype(str).sample(sample_n, random_state=42)
X_cnt = vec.fit_transform(sample_texts)
term_freq = np.asarray(X_cnt.sum(axis=0)).ravel()
vocab = np.array(vec.get_feature_names_out())
top_k = 30
top_idx = term_freq.argsort()[-top_k:][::-1]
top_terms = vocab[top_idx]
top_vals = term_freq[top_idx]

plt.figure(figsize=(10,6))
plt.bar(range(top_k), top_vals)
plt.xticks(range(top_k), top_terms, rotation=75, ha="right")
plt.title(f"Top {top_k} frequent terms (sample size={sample_n:,})")
plt.xlabel("term")
plt.ylabel("freq")
plt.tight_layout()
plt.savefig(OUT_DIR / "top_terms_bar.png", dpi=150)
plt.close()

# ---------- 6) پراکنش دوبعدی (2D) از ویژگی‌ها ----------
# دو راه:
#   الف) اگر NPZ کم‌بعدی (SRP) داری: روی نمونه‌ای از آن SVD دو مؤلفه‌ای بزن.
#   ب) اگر NPZ نداری: همین CountVectorizer بالا را با TruncatedSVD به 2 بُعد ببَر (کیفیت پایین‌تر، ولی دید می‌دهد).

plot_saved = False
if Path(VEC_NPZ).exists():
    # لود ویژگی‌های کم‌بعدی (مثلاً 256 بُعد)
    X_all = sparse.load_npz(VEC_NPZ)  # CSR (n_samples, 256)
    n_samples = min(40_000, X_all.shape[0])
    rng = np.random.default_rng(42)
    idx = rng.choice(X_all.shape[0], size=n_samples, replace=False)
    X_s = X_all[idx]

    svd2 = TruncatedSVD(n_components=2, random_state=42)
    X2 = svd2.fit_transform(X_s)  # (n_samples, 2)

    plt.figure(figsize=(7,6))
    plt.scatter(X2[:,0], X2[:,1], s=3, alpha=0.35)
    plt.title(f"2D projection of SRP features (n={n_samples:,})")
    plt.xlabel("SVD-1")
    plt.ylabel("SVD-2")
    plt.tight_layout()
    plt.savefig(OUT_DIR / "svd_scatter_from_SRP.png", dpi=150)
    plt.close()
    plot_saved = True

if not plot_saved:
    # fallback: روی همان X_cnt (CountVectorizer) 2D بگیر (نمونه کوچک‌تر)
    svd2 = TruncatedSVD(n_components=2, random_state=42)
    X2 = svd2.fit_transform(X_cnt)  # (sample_n, 2)

    plt.figure(figsize=(7,6))
    plt.scatter(X2[:,0], X2[:,1], s=3, alpha=0.35)
    plt.title(f"2D projection from CountVectorizer (n={sample_n:,})")
    plt.xlabel("SVD-1")
    plt.ylabel("SVD-2")
    plt.tight_layout()
    plt.savefig(OUT_DIR / "svd_scatter_from_Count.png", dpi=150)
    plt.close()

print("✅ تصاویر در این پوشه ذخیره شدند:", OUT_DIR)


✅ تصاویر در این پوشه ذخیره شدند: D:\code\projects\EmoLex(امولکس)\Data\دیجی کالا\viz
